In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import xgboost as xgb
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())
import datetime
from sklearn.cross_validation import train_test_split
import operator
from scipy.stats import *

0it [00:00, ?it/s]
/Library/Python/2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
train = pd.read_csv('./train_1.csv').fillna(0)

In [58]:
key = pd.read_csv('./key_trunc.csv')

In [59]:
sub = pd.read_csv("./sample_submission_1.csv")

In [389]:
train_melt = pd.read_csv("train_feats.csv")

In [234]:
for col in train.columns[1:]:
    train[col] = pd.to_numeric(train[col],downcast='integer')

KeyboardInterrupt: 

In [511]:
def get_median(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).median()[rows_to_consider - 1:])

In [512]:
def get_mean(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider:]) 
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).mean()[rows_to_consider - 1:])

In [514]:
def get_std(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).std()[rows_to_consider - 1:])

In [513]:
def get_min(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).min()[rows_to_consider - 1:])        

In [515]:
def get_max(row, last_n_days, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider:])
    else:
        return pd.Series(pd.Series(row)[1:].rolling(min_periods=7,window=last_n_days,center=False).max()[rows_to_consider - 1:])        

In [528]:
def get_mean_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].mean()).repeat(len(row) - rows_to_consider)

In [529]:
def get_median_overall(row, rows_to_consider, is_it_train):
    if is_it_train == 1:
        return pd.Series(pd.Series(row)[1:].median()).repeat(-1*rows_to_consider)
    else:
        return pd.Series(pd.Series(row)[1:].median()).repeat(len(row) - rows_to_consider)

In [461]:
# def get_mean_diff(row, last_n_days):
#     return pd.Series(pd.rolling_mean(np.diff(pd.Series(row)), window = last_n_days))

In [462]:
# def get_last_day_visits(row):
#     return row[1:]

In [520]:
def get_long_stats(func_type, df, df_melt, rows_to_consider, is_it_train = None, last_n_days = None):

    if func_type == "median":
        rolling_stats = df.apply(get_median, last_n_days = last_n_days, rows_to_consider = rows_to_consider,is_it_train = is_it_train, axis = 1)
        
    elif func_type == "mean":
        rolling_stats = df.apply(get_mean, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
    
    elif func_type == "std":
        rolling_stats = df.apply(get_std, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "min":
        rolling_stats = df.apply(get_min, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "max":
        rolling_stats = df.apply(get_max, last_n_days = last_n_days, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "mean_overall":
        rolling_stats = df.apply(get_mean_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)

    elif func_type == "median_overall":
        rolling_stats = df.apply(get_median_overall, rows_to_consider = rows_to_consider, is_it_train = is_it_train, axis = 1)
        
    rolling_stats_df = pd.concat([df.Page, rolling_stats.apply(pd.Series)], axis = 1)  
    rolling_stats_df.columns = ["Page"] + list(df.columns[rows_to_consider:])
    
    rolling_stats_df = pd.melt(rolling_stats_df, id_vars=['Page'], var_name="date", value_name=func_type + "_" + str(last_n_days))        
    
    df_melt = pd.merge(df_melt, rolling_stats_df, on = ["Page", "date"])     
    
    return df_melt

In [521]:
def generate_train_melt(train):
    
    train_melt = pd.melt(pd.concat([train.Page, 
                                    train.iloc[:, -60:]],
                                    axis = 1),
                                    id_vars=['Page'], 
                                    var_name="date", value_name="visits")

    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("mean", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)

    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("median", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)

    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("std", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)

    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("min", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)

    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=30)
    train_melt = get_long_stats("max", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)

    train_melt = get_long_stats("mean_overall", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    train_melt = get_long_stats("median_overall", train, train_melt, rows_to_consider=-60, is_it_train = 1, last_n_days=60)
    
    
    train_melt["date_num"] = train_melt.apply(get_date, axis = 1)
    train_melt["weekday"] = train_melt.apply(get_weekday, axis = 1)
    
    return train_melt 

def get_lang_features(train_melt):
    
    train_melt = pd.merge(train_melt, pd.concat([train.Page, train.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    train_melt.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = train_melt[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

    lang_feats.columns = ["language", "date", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean", "lang_min_30_mean", "lang_min_60_mean", "lang_max_30_mean", "lang_max_60_mean"]
        
    train_melt = pd.merge(train_melt, lang_feats, on = ["language", "date"])
    
    return train_melt

def get_weekday_features(train_melt):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    train_melt = pd.merge(train_melt, weekday_feats, on = ["weekday", "Page"])
    
    return train_melt
    
# def get_lang_mean(train_melt):
    
#     language_page_melt = train_melt[["language", "date", "visits"]]
    
#     language_feats = pd.DataFrame(language_page_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))

#     language_feats.columns = ["language", "date", "lang_mean_visits"]
        
#     train_melt = pd.merge(train_melt, language_feats, on = ["language", "date"])
    
#     return train_melt

train_melt = generate_train_melt(train)
train_melt = get_lang_features(train_melt)
train_melt = get_weekday_features(train_melt)
train_melt.to_csv("train_feats.csv", index = False)

,Page,date,visits,mean_30,mean_60,median_30,median_60,std_30,std_60,min_30,min_60,max_30,max_60,mean_overall_60,median_overall_60,date_num,weekday
0,2NE1_zh.wikipedia.org_all-access_spider,2016-11-02,18.0,19.000000,21.066667,16.0,17.5,10.218307,13.528209,8.0,8.0,50.0,72.0,21.756364,16.0,02,2
1,2PM_zh.wikipedia.org_all-access_spider,2016-11-02,11.0,39.933333,35.983333,26.0,25.5,38.361199,30.086030,11.0,11.0,179.0,179.0,25.392727,17.0,02,2
2,3C_zh.wikipedia.org_all-access_spider,2016-11-02,3.0,4.066667,4.983333,4.0,4.0,2.242741,3.270265,0.0,0.0,11.0,19.0,5.203636,4.0,02,2
3,4minute_zh.wikipedia.org_all-access_spider,2016-11-02,12.0,19.066667,19.233333,14.0,14.5,18.339581,17.188256,4.0,4.0,103.0,103.0,17.125455,13.0,02,2
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2016-11-02,5.0,13.833333,10.350000,7.0,6.0,30.440539,22.998029,2.0,0.0,173.0,173.0,4.840000,0.0,02,2
5,2NE1_zh.wikipedia.org_all-access_spider,2016-11-03,25.0,19.366667,21.166667,17.0,17.5,10.230055,13.534847,8.0,8.0,50.0,72.0,21.756364,16.0,03,3
6,2PM_zh.wikipedia.org_all-access_spider,2016-11-03,14.0,39.800000,34.916667,26.0,25.0,38.446918,29.703217,11.0,11.0,179.0,179.0,25.392727,17.0,03,3
7,3C_zh.wikipedia.org_all-access_spider,2016-11-03,3.0,3.966667,5.000000,3.5,4.0,2.220334,3.257326,0.0,0.0,11.0,19.0,5.203636,4.0,03,3
8,4minute_zh.wikipedia.org_all-access_spider,2016-11-03,11.0,18.866667,19.150000,13.0,14.0,18.395527,17.216296,4.0,4.0,103.0,103.0,17.125455,13.0,03,3
9,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,2016-11-03,6.0,13.900000,10.383333,7.0,6.0,30.420445,22.990117,2.0,0.0,173.0,173.0,4.840000,0.0,03,3


In [93]:
def get_source(row):
    return row.Page.split["."][-1].split("_")[-1]

In [94]:
def get_language(row):
    try:
        return row.Page.split(".")[0].split("_")[-1]    
    except:
        return "en"

In [95]:
def get_month(row):
    return row.date.split("-")[1]

In [96]:
def get_date(row):
    return row.date.split("-")[2]

In [97]:
def get_weekday(row):
    return datetime.datetime.strptime(row.date, '%Y-%m-%d').date().weekday()

In [13]:
# def get_language_mean(row):
# lang_count = pd.Series(train_melt.groupby(["language"])["Page"].count()).sort_values(axis = 0, ascending = False)
# print lang_count

In [66]:
def get_lang_features_test(test_df):
    
    test_df = pd.merge(test_df, pd.concat([overall_df.Page, overall_df.progress_apply(get_language, axis = 1)], axis = 1), on = "Page")
    test_df.columns = ['Page', 'date', 'visits', 'mean_30', 'mean_60', 'median_30', 'median_60', 'std_30', 'std_60', 'min_30', 'min_60', 'max_30', 'max_60', 'mean_overall', 'median_overall', 'date_num', 'weekday', 'language']
    
    lang_date_melt = test_df[["language", "date", "mean_30", "mean_60", "median_30", "median_60", "min_30", "min_60", "max_30", "max_60"]]
    
    lang_feats = pd.DataFrame(lang_date_melt.groupby(["language", "date"], as_index = False).aggregate(np.mean))
    lang_feats.columns = ["language", "date", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean", "lang_min_30_mean", "lang_min_60_mean", "lang_max_30_mean", "lang_max_60_mean"]
        
    test_df = pd.merge(test_df, lang_feats, on = ["language", "date"])
    
    return test_df

In [409]:
def get_weekday_features_test(train_melt, test_df):
    
    weekday_page_melt = train_melt[["weekday", "Page", "visits"]]
    
    weekday_mean_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.mean))
    weekday_mean_feats.columns = ["weekday", "Page", "weekday_mean_visits"]
    
    weekday_median_feats = pd.DataFrame(weekday_page_melt.groupby(["weekday", "Page"], as_index = False).aggregate(np.median))
    weekday_median_feats.columns = ["weekday", "Page", "weekday_median_visits"]
    
    weekday_feats = pd.merge(weekday_mean_feats, weekday_median_feats, on = ["weekday", "Page"])
        
    train_melt = pd.merge(test_df, weekday_feats, on = ["weekday", "Page"])
    
    return train_melt

In [148]:
# def get_last_day_visits_test(row, last_index):
    
#     return row.iloc[last_index - 1]

In [219]:
def smape(y_pred, y_true):
        
    y_true = y_true.get_label()
    y_pred = np.array(y_pred)
        
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0

    return "smape", np.nanmean(diff)

In [361]:
def run_xgb(train, label_train, valid = None, label_valid = None):

    # Set our parameters for xgboost
    params = {}
    params['objective'] = 'reg:linear'
    params['eta'] = 0.05
    params['max_depth'] = 7
    params['silent'] = 1
    params['min_child_weight'] = 0
    params['subsample'] = 1
    params['colsample_bytree'] = 1
    params['nthread'] = 13
    params['gamma'] = 0
    params['max_delta_step'] = 0
    
    d_train = xgb.DMatrix(train, label=label_train)
    
    if valid is not None:
        d_valid = xgb.DMatrix(valid, label=label_valid)
        watchlist = [(d_train, 'train'), (d_valid, 'validation')]
    else:
        watchlist = [(d_train, 'train')]
        
    bst = xgb.train(params, d_train, 100, watchlist, early_stopping_rounds=10, verbose_eval=10, feval=smape, maximize = False)

    return bst

In [364]:
# label_train_melt = np.log1p(train_melt["visits"])
label_train_melt = train_melt["visits"]

# label_train_melt = boxcox(np.array(label_train_melt), lmbda=0.094) #lambda found from default boxcox on outlier-stripped data

# label_train_melt = [0] + np.diff(train_melt["visits"]).tolist()

train_melt.drop("visits", axis = 1, inplace = True)
train_melt.drop("date", axis = 1, inplace = True)
train_melt.drop("language", axis = 1, inplace = True)
train_melt.drop("Page", axis = 1, inplace = True)

# train_melt.drop("last_day_visits_None", axis = 1, inplace = True)
# train_melt.drop("mean_diff_7", axis = 1, inplace = True)

# train_melt.drop("mean_7", axis = 1, inplace = True)
# train_melt.drop("median_7", axis = 1, inplace = True)

train_melt.date_num = pd.to_numeric(train_melt.date_num)

KeyError: 'visits'

In [366]:
x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
bst = run_xgb(x_train, label_train, x_valid, label_valid)

# bst = run_xgb(train_melt, label_train_melt)

[0]	train-rmse:74811.9	validation-rmse:93642.3	train-smape:166.269	validation-smape:165.439
Multiple eval metrics have been passed: 'validation-smape' will be used for early stopping.

Will train until validation-smape hasn't improved in 10 rounds.
[10]	train-rmse:61550.3	validation-rmse:71279.5	train-smape:78.2561	validation-smape:78.261
[20]	train-rmse:55873.4	validation-rmse:60720.7	train-smape:54.4201	validation-smape:54.8868
[30]	train-rmse:53610.8	validation-rmse:56257.2	train-smape:47.8532	validation-smape:48.1546
[40]	train-rmse:52659	validation-rmse:54191.3	train-smape:46.4056	validation-smape:46.6586
[50]	train-rmse:52247.1	validation-rmse:53272.5	train-smape:46.718	validation-smape:47.2014
Stopping. Best iteration:
[43]	train-rmse:52501	validation-rmse:53831.7	train-smape:46.2859	validation-smape:46.6444



In [367]:
importance = bst.get_fscore()

importance = sorted(importance.items(), key=operator.itemgetter(1))

df_imp = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_imp['fscore'] = df_imp['fscore']/df_imp['fscore'].sum()

print df_imp

                  feature    fscore
0     lang_median_30_mean  0.008225
1                  min_60  0.012479
2       lang_mean_30_mean  0.013897
3        lang_min_60_mean  0.014464
4     lang_median_60_mean  0.015031
5        lang_min_30_mean  0.016166
6        lang_max_30_mean  0.020987
7                 mean_60  0.022405
8        lang_max_60_mean  0.024107
9                 weekday  0.025241
10              median_60  0.029779
11         median_overall  0.032898
12      lang_mean_60_mean  0.034317
13                 std_60  0.036869
14                 max_60  0.042825
15                 min_30  0.043676
16  weekday_median_visits  0.048780
17                 std_30  0.049348
18           mean_overall  0.055303
19               date_num  0.055871
20              median_30  0.064663
21                mean_30  0.087067
22                 max_30  0.120533
23    weekday_mean_visits  0.125071


In [245]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

# x_train, x_valid, label_train, label_valid = train_test_split(train_melt, label_train_melt, test_size=0.2, random_state=4242)
# # x_train = x_train.fillna(0)
# # x_valid = x_valid.fillna(0)
# reg.fit(x_train, label_train)

reg.fit(train_melt, label_train_melt)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [243]:
# prediction = reg.predict(x_valid)
prediction = reg.predict(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

In [244]:
y_true = np.array(label_valid)
y_pred = np.array(prediction)

denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
diff = np.abs(y_true - y_pred) / denominator
diff[denominator == 0] = 0.0

print np.nanmean(diff)

105.106730814


In [229]:
def get_preds(test_df):
    
    d_test = xgb.DMatrix(test_df)
    p_test = bst.predict(d_test)
    
    return pd.Series(p_test)

In [440]:
def get_overall_df():
    
    key_wide = pd.DataFrame()
    key_wide["page"] = np.unique(key.trunc_page).tolist()
    for i in np.unique(key.date).tolist():
        key_wide[i] = np.nan

    overall_df = pd.merge(train, key_wide, left_on="Page", right_on = "page", how = "inner")
    overall_df.drop("page", axis = 1, inplace = True)
    
    return overall_df

In [530]:
def generate_test_melt(overall_df):
        
    pred_index = overall_df.columns.tolist().index('2017-01-01') 
    len_columns = len(overall_df.columns)
    
    test_melt = pd.melt(pd.concat([overall_df.Page, overall_df.iloc[:, pred_index:]],axis = 1),
                                id_vars=['Page'], 
                                var_name="date", value_name="visits")
    
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("mean", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("median", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("std", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("min", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 30)
    test_melt = get_long_stats("max", overall_df, test_melt, pred_index, is_it_train=0, last_n_days = 60)

    test_melt = get_long_stats("mean_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    test_melt = get_long_stats("median_overall", overall_df, test_melt, rows_to_consider = pred_index, is_it_train=0, last_n_days = 60)
    
    test_melt["date_num"] = test_melt.apply(get_date, axis = 1)
    test_melt["weekday"] = test_melt.apply(get_weekday, axis = 1)

    return test_melt 

In [531]:
overall_df = get_overall_df()
test_melt = generate_test_melt(overall_df)
test_df = get_lang_features_test(test_df)
test_df = get_weekday_features_test(train_melt, test_df)
test_df.drop("visits", axis = 1, inplace=True)
test_df.to_csv("test_feats.csv", index = False)

In [230]:
test_df = pd.read_csv("test_feats.csv")

In [231]:
prediction = get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday", "lang_mean_30_mean", "lang_mean_60_mean", "lang_median_30_mean", "lang_median_60_mean"]])

# prediction = invboxcox(get_preds(test_df[["mean_30","mean_60","median_30","median_60","date_num", "weekday"]]), ld = 0.094)

In [240]:
# pred_index = overall_df.columns.tolist().index("2017-01-01")

# pred_df = overall_df.iloc[:, pred_index:]
# pred_df["Page"] = overall_df.Page

# pred_df_melt = pd.melt(pred_df, id_vars=['Page'], var_name="date", value_name="Visits")        

test_df["Visits"] = prediction
# pred_df = pd.concat([test_df[["Page", "date"]], prediction], axis = 1)
pred_df = test_df[["Page", "date", "Visits"]]

pred_id_visits = pd.merge(pred_df, key, left_on=["Page", "date"], right_on=["trunc_page", "date"], how="inner")[["Id", "Visits"]]

pred_id_visits.to_csv("pred_glm_3.csv", index = False)

In [7]:
from scipy.stats import *
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt if i > 4 and i < 1871])) #Outlier 10% to 90% only
# bc_label, lambda_val = boxcox(np.array([i for i in label_train_melt]))

bc_label = boxcox(np.array(label_train_melt), lmbda=0.094)

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/scipy/stats/morestats.py:516: RuntimeWarning: divide by zero encountered in log
  y = where(lmbda == 0, log(x), (x**lmbda - 1)/lmbda)


In [8]:
def invboxcox(y,ld):
    if ld == 0:
        return(np.exp(y))
    else:
        return(np.exp(np.log(ld*y+1)/ld))

In [10]:
# np.percentile(label_train_melt, 90)
# np.percentile(label_train_melt, 10)
# len([i for i in label_train_melt if i > 4 and i < 1871])

# import matplotlib.pyplot as plt
# plt.hist(bc_label.tolist())
# plt.show()

# bc_label

# import scipy.special.inv_boxcox
invboxcox(bc_label, ld = 0.094)

/Library/Python/2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


array([ 18.,  11.,   3., ...,   0.,   0.,   0.])